In [15]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as img
import tensorflow.keras as keras
import numpy as np
import tensorflow as tf
import pandas as pd
import sklearn
import cv2

from sklearn.cluster import KMeans
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
from rembg import remove

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import keras.utils as image
from keras.applications.vgg16 import preprocess_input

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

Modelo de para Clasificar Ropa en los distintos Labels

In [16]:
#cambiar directorio de ser necesario
datasetdir = r'.\Train'
os.chdir(datasetdir)
batch_size = 5

FileNotFoundError: [WinError 2] The system cannot find the file specified: '.\\Train'

In [ ]:
#Funcion para cargar data y clasificarla en data para entrenamiento (%90) y para validacion (%10)
def DataLoad(shape, preprocessing): 
    
    #aqui se agrega los parametros para realizar Data Augmentation
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.15, zoom_range=0.1, 
        channel_shift_range=10., horizontal_flip=True,
        validation_split = 0.1,
    )

    height, width = shape

    #Se declaran los 17 labels de clasificacion y sus dimensiones
    train_dataset = imgdatagen.flow_from_directory(
        os.getcwd(),
        target_size = (height, width), 
        classes = ['abrigo', 'blazer', 'blusa', 'boxer', 'camisa',
                  'chaleco', 'enterizo', 'falda', 'joggers', 'legging',
                  'medias', 'pantalon', 'panty', 'short', 'sosten', 'trajedebaño', 'vestido'],
        batch_size = batch_size,
        subset = 'training', 
    )

    val_dataset = imgdatagen.flow_from_directory(
        os.getcwd(),
        target_size = (height, width), 
        classes = ['abrigo', 'blazer', 'blusa', 'boxer', 'camisa',
                  'chaleco', 'enterizo', 'falda', 'joggers', 'legging',
                  'medias', 'pantalon', 'panty', 'short', 'sosten', 'trajedebaño', 'vestido'],
        batch_size = batch_size,
        subset = 'validation'
    )
    return train_dataset, val_dataset

In [ ]:
#se carga la red vgg16 
vgg16 = keras.applications.vgg16

#usamos nuestra funcion dataload para cargar nuestra imagenes y procesarlas para que encaje con el modelo pre procesado
train_dataset, val_dataset = DataLoad((224,224), preprocessing=vgg16.preprocess_input)
x_train, y_train = next(train_dataset)

#obtenemos el modelo convolucional vgg16, sin incluir las capas finales de clasificacion
conv_model = vgg16.VGG16(weights='imagenet', include_top=False)
conv_model.summary()

In [ ]:
#convertimos en modelo de multiples dimensiones a un arreglo de una dimension
x = keras.layers.Flatten()(conv_model.output)

#agregamos 3 capas fully conected de con 100 neuronas con funcion de activiacion relu 
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)

#finalmente una funcion softmax para que clasifique la respuesta a uno de los 17 labels
predictions = keras.layers.Dense(17, activation='softmax')(x)

# creacion del modelo completo
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
full_model.summary()

In [ ]:
#aseguramos que las capas de la vgg16 no sean entrenadas 
for layer in conv_model.layers:
    layer.trainable = False

In [ ]:
#compilamos el modelo
#la funcion de costo categorical crossentropy porque son multiples clases excluyentes entre si
#optimizaremos con learning rate muy pequeño y adamax
full_model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adamax(learning_rate=0.001),
                  metrics=['acc'])

In [ ]:
#entrenamos nuestro modelo
#las epocas se ajustan dependiendo de los resultados
history = full_model.fit(
    train_dataset, 
    validation_data = val_dataset,
    workers=0,
    epochs=8,
)

In [ ]:
#guardamos el modelo
full_model.save("modelo_CR.h5")

In [ ]:
#evaluamos nuestro modelo con las imagenes de Test
test_dir= r'.\Test'
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=3, class_mode='categorical')


Testresults = full_model.evaluate(test_generator)

In [ ]:

#para pruebas cambiamos el directorio a la imagen que se quiere probar
img_path = r'.\Test\short\short-milen.jpg'

#se preprocesa la imagen
img = image.load_img(img_path, target_size=(224,224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

#se la pasa por el modelo para obtener una prediccion
imagen_prediccion=full_model.predict(x)

#visualizar imagen
plt.imshow(img)

In [ ]:
#funcion para obtener el nombre de la clase
def obtener_nombre_clase(index):
   for class_string, class_index in test_generator.class_indices.items():
      if class_index == index:
         return class_string

#indice de imagen    
indice_clase_prediccion = np.argmax(imagen_prediccion, axis = 1)
print('indice predecido es:', indice_clase_prediccion) 

#label a usarse en busqueda
label_prediccion = obtener_nombre_clase(clase_prediccion)
print("label: " + label_prediccion)

Quitar Background a Imagenes

In [ ]:
#ruta donde esta la imagen
input_path = r'..\Test\short\short-milen.jpg'
#ruta donde se guardara imagen
output_path = r'..\Test\short\sinbg-short-milen2.png'
input = Image.open(input_path)
output = remove(input)
output.save(output_path)

Clasificación de Colores

In [17]:
#funcion para obtener imagen
def get_imagen(imagen_path):
    imagen = cv2.imread(imagen_path)
    imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
    return imagen

#funcion para convertir a hexadecimal    
def RGB2HEX(color):
    
    return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

In [9]:
#declarar ruta de imagen
imagen = get_imagen(r'..\Test\short\sinbg-short-milen.png')
#el numero de colores del resultado
num_colores = 5
resized_imagen = imagen.reshape(imagen.shape[0]*imagen.shape[1], 3)

#clasificacion por clusters de los colores
clf = KMeans(n_clusters = num_colores)
labels = clf.fit_predict(resized_imagen)

In [18]:
#transformacion de resultado de labels a hex y rgb
num_labels = Counter(labels)

center_colors = clf.cluster_centers_


ordered_colors = [center_colors[i] for i in num_labels.keys()]

#arreglo de resultado
hex_colors = [RGB2HEX(ordered_colors[i]) for i in num_labels.keys()]
rgb_colors = [ordered_colors[i] for i in num_labels.keys()]
#para obtener rgb en ints - (int(color[0]), int(color[1]), int(color[2]))


3 3 3
202 202 202
137 137 136
84 83 82
173 172 172
[array([3.46947988, 3.42120151, 3.32310819]), array([202.24312186, 202.30008017, 202.92633735]), array([137.67199694, 137.22651679, 136.12278098]), array([84.19007587, 83.4740885 , 82.31432216]), array([173.08096683, 172.57837033, 172.23625566])]
